In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('data/dog_adopt_eda.csv')

### 건강 컬럼 LLM 사용하여 질병 컬럼으로 생성   
질병 컬럼 값 : 유, 무

In [ ]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from dotenv import load_dotenv

load_dotenv()

llm_model = ChatOpenAI(model='gpt-4o', temperature=0 )

health_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template('''
                                                  입력 값은 유기견의 건강상태야. 
                                                  입력된 값에 질병이나 건강에 대한 부정적인 말이 작성되어 있다면 '질병' 이라고 표시해.
                                                  아닌 것들은 건강으로 표시해
                                                  출력 :
                                                  건강
                                                  위의 형식으로 출력해줘.
                                                  '''),
        HumanMessagePromptTemplate.from_template('{input}')
    ]
)

health_chain = LLMChain(
    llm = llm_model,
    prompt = health_prompt
)

C:\Users\Admin\AppData\Local\Temp\ipykernel_26232\4081578801.py:8: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm_model = ChatOpenAI(model='gpt-4o', temperature=0 )
C:\Users\Admin\AppData\Local\Temp\ipykernel_26232\4081578801.py:24: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(


In [ ]:
health = []

for h in df['건강']:
    result = health_chain.invoke(h)['text']
    health.append(result)

health

KeyError: '건강'

In [ ]:
df['질병'] = health

In [ ]:
df['질병'].value_counts()

# gpt 출력 결과에서 필요 없는 정보 삭제
# df['질병'] = df['질병'].str.replace(r'출력 :  \n', '', regex=True)
df.drop(columns=['건강'], inplace=True)

질병
건강    1409
질병     459
Name: count, dtype: int64

### 색 컬럼 LLM 사용하여 세가지 계열로 분류
색 : 밝은계열, 어두운계열, 혼합색계열

In [ ]:
color_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template('''
                                                    입력 값은 유기견의 색이야. 
                                                    입력된 값이 밝은 계열의 색이면 밝은계열이라고 표시해.
                                                    입력된 값이 어두운 계열의 색이면 어두운계열이라고 표시해.
                                                    입력된 값이 혼합색 계열의 색이면 혼합색계열이라고 표시해.
                                                    출력 :
                                                    밝은계열  
                                                    위의 형식처럼 하나의 계열만 출력해줘.
                                                  '''),
        HumanMessagePromptTemplate.from_template('{input}')
    ]
)

color_chain = LLMChain(
    llm = llm_model,
    prompt = color_prompt
)

In [ ]:
color = []

for h in df['색']:
    result = color_chain.invoke(h)['text']
    color.append(result)

df['색'] = color

In [21]:
df['색'].value_counts()

색
밝은계열     1090
혼합색계열     480
어두운계열     298
Name: count, dtype: int64

### 성격 컬럼 LLM 사용하여 세가지 값으로 분류
성격: 사람을 잘 따름, 사람을 경계함, 중립

In [ ]:
character_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template('''
                                                    입력 값은 유기견의 성격이야. 
                                                    입력된 값이 사람과 잘 어울리는 성격 계열이면 '사람을 잘 따름'이라고 표시해.
                                                    입력된 값이 사람과 잘 어울리지 못하는 성격 계열이면 '사람을 경계함'이라고 표시해.
                                                    입력된 값이 판별하기 어려운 성격 계열이면 '중립'이라고 표시해.
                                                    입력된 값이 없으면 '중립'이라고 표시해.

                                                    출력 :
                                                    사람을 잘 따름
                                                    위의 형식으로 출력해줘.
                                                  '''),
        HumanMessagePromptTemplate.from_template('{input}')
    ]
)

character_chain = LLMChain(
    llm = llm_model,
    prompt = character_prompt
)

In [ ]:
character = []

for h in df['성격']:
    result = character_chain.invoke(h)['text']
    character.append(result)

df['성격'] = character

In [47]:
df['성격'].value_counts()

성격
중립          1084
사람을 잘 따름     489
사람을 경계함      295
Name: count, dtype: int64

### 성별 컬럼 결측치(미상) 삭제

In [ ]:
index = df['성별'][df['성별']=='미상'].index
index

Index([52, 167, 547, 647, 648, 649, 650, 651, 652, 653, 654, 848, 1059], dtype='int64')

In [ ]:
df['성별'].value_counts()

성별
암컷    955
수컷    900
Name: count, dtype: int64

In [36]:
df['성별'].value_counts()

성별
암컷    955
수컷    900
미상     13
Name: count, dtype: int64

### 품종 rare 값을 품종견으로 분류
품종: 믹스견, 품종견

In [ ]:
df['품종'][~(df['품종']=='믹스견')] = '품종견'

C:\Users\Admin\AppData\Local\Temp\ipykernel_26232\1034341778.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df2['품종'][~(df2['품종']=='믹스견')] = '품종견'
C:\Users\Admin\AppData\Local\Temp\ipykernel_26232\1034341778.py:1: SettingWithCopyWarning

In [ ]:
df['품종'].value_counts()

품종
믹스견    1410
품종견     445
Name: count, dtype: int64

### csv로 저장

In [ ]:
df.to_csv('data/dog_final_1203.csv', encoding='utf-8-sig', index=False)